In [84]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [85]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Đường dẫn đến file PDF của bạn
pdf_path = "data/LSĐ.pdf" # Thay thế bằng đường dẫn thực tế

# Tải tài liệu
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Chia nhỏ văn bản thành các đoạn nhỏ hơn để dễ dàng nhúng
text_splitter = RecursiveCharacterTextSplitter(
    separators="\n",
    chunk_size=1024,      # Kích thước tối đa của mỗi đoạn
    chunk_overlap=256,    # Số ký tự trùng lặp giữa các đoạn
    length_function=len,
    add_start_index=True,
)
texts = text_splitter.split_documents(documents)

In [59]:
print(f"Số lượng đoạn văn sau khi chia nhỏ: {len(texts)}")
print(texts[1]) # In ra đoạn văn đầu tiên để kiểm tra

Số lượng đoạn văn sau khi chia nhỏ: 707
page_content='mạnh mẽ, rộng khắp, nhất là ở châu Á. Cùng với  phong trào đấu tranh của giai cấp vô sản chống 
lại giai cấp tư sản ở các nước tư bản chủ  nghĩa, phong trào giải phóng dân tộc ở các nước thuộc 
địa trở thành một bộ phận quan  trọng trong cuộc đấu tranh chung chống tư bản, thực dân. Phong 
trào giải phóng dân tộc ở các nước châu Á đầu thế kỷ XX phát triển rộng khắp, tác động mạnh mẽ  
đến phong trào yêu nước  Việt Nam. 
Trong bối cảnh đó, thắng lợi của Cách mạng Tháng Mười Nga năm 1917 đã làm biến 
đổi sâu sắc tình hình thế giới. Thắng lợi của Cách mạng Tháng Mười Nga không chỉ có ý nghĩa to 
lớn đối với cuộc đấu tranh của giai cấp vô sản đối với các nước tư bản, mà  còn có tác động sâu 
sắc đến phong trào giải phóng dân tộc ở các thuộc địa. Tháng 3 -1919, Quốc tế Cộng sản, do 
V.I.Lênin đứng đầu, được thành lập, trở thành bộ tham mưu chiến  đấu, tổ chức lãnh đạo phong 
trào cách mạng vô sản thế giới. Quốc tế Cộng sản không  những vạ

In [86]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Khởi tạo mô hình embeddings
# device='cpu' đảm bảo nó chạy trên CPU
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

print("Mô hình embeddings đã được tải thành công trên CPU.")

Mô hình embeddings đã được tải thành công trên CPU.


In [33]:
# Lấy nội dung của texts[1]
text_to_embed = texts[1].page_content

# Nhúng đoạn văn bản này
embedded_vector = embeddings_model.embed_query(text_to_embed)

# In ra vector nhúng
print("\nVector nhúng của texts[1]:")
print(embedded_vector)

# In ra kích thước của vector (để xác nhận)
print(f"\nKích thước của vector nhúng: {len(embedded_vector)}")


Vector nhúng của texts[1]:
[-0.0473400354385376, -0.003674556501209736, 0.03247338533401489, -0.08391741663217545, -0.07184924185276031, -0.0073947785422205925, 0.05632589012384415, -0.014187678694725037, 0.06621778011322021, 0.041439276188611984, 0.1420973837375641, -0.1105559766292572, -0.053706295788288116, -0.014503752812743187, 0.0027634799480438232, -0.03194497525691986, -0.07741059362888336, 0.04760720953345299, -0.06908846646547318, -0.07517044246196747, -0.028736498206853867, 0.03211483359336853, -0.0338798388838768, 0.016187310218811035, -0.061498478055000305, 0.029214654117822647, 0.011894816532731056, 0.08688092976808548, 0.011595932766795158, -0.03737006336450577, -0.010598300956189632, 0.14106853306293488, 0.007223378401249647, -0.011536149308085442, 0.04257099702954292, 0.053089890629053116, -0.005555435083806515, -0.0678093284368515, 0.058464981615543365, 0.014127600938081741, -0.03976449742913246, -0.041157837957143784, 0.02193848043680191, -0.06027468293905258, 0.048

In [87]:
from langchain_chroma import Chroma

# Đặt tên cho database vector
db_name = "chroma_db"

# kiểm tra nếu database chorma đã tồn tại, thì xóa collection để khởi động lại từ đầu
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings_model).delete_collection()

# Tạo một vectorstore từ các đoạn văn bản và embeddings
# Đây sẽ là nơi lưu trữ dữ liệu để truy vấn RAG
vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embeddings_model,
    persist_directory=db_name   # Lưu trữ vào thư mục này để có thể sử dụng lại
)

# Lưu trữ vectorstore (quan trọng để không phải tạo lại mỗi lần)
# vectorstore.persist()
print("Dữ liệu đã được lưu trữ vào Chroma DB.")

# Kiểm tra số lượng document đã được lưu vào vector store
print(f"Vectorstore đã được tạo với {vectorstore._collection.count()} documents")

# Để tải lại từ thư mục đã lưu:
# vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings_model)

Dữ liệu đã được lưu trữ vào Chroma DB.
Vectorstore đã được tạo với 707 documents


In [88]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
# Khởi tạo LLM Gemini Pro
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.5,
)

# Khởi tạo memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Định nghĩa prompt cho chatbot
# Sử dụng placeholder {context} và {question}
custom_prompt_template = """Bạn là một trợ lí AI hữu ích và lịch sự.
Sử dụng các đoạn ngữ cảnh sau đây để trả lời câu hỏi ở cuối.
Nếu bạn không biết câu trả lời, hãy nói rằng bạn không biết, đừng cố bịa ra câu trả lời.

Ngữ cảnh: {context}
Câu hỏi: {question}

Câu trả lời:"""

# Tạo PromptTemplate
PROMPT = PromptTemplate(
    template=custom_prompt_template,
    input_variables=["context", "question"]
)

In [63]:
from langchain.chains import RetrievalQA

# Thích hợp cho các ứng dụng hỏi đáp một lần, nơi mỗi câu hỏi là riêng biệt và không cần ngữ cảnh từ các cuộc trò chuyện trước.

db_name = "Chorma_db_01"

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings_model).delete_collection()

texts = ["Hưng Đạo là một trong những vị danh tướng kiệt xuất nhất lịch sử Việt Nam.",
         "Phạm Ngũ Lão ra và lớn lên trong thời kì đầy khói lửa của chiến tranh.",
         "Phạm Ngũ Lão có công rất lớn trong việc xây dựng, bảo vệ, củng cố sức mạnh dưới thời nhà Trần."]

vectorstore_01 = Chroma.from_texts(
    texts=texts,
    embedding=embeddings_model,
    persist_directory=db_name   
)
print(f"Vectorstore đã được tạo với {vectorstore_01._collection.count()} documents")

retriever = vectorstore_01.as_retriever()

# Tạo RetrievalQA chain
# Thiết lập return_source_documents=True để xem nguồn tài liệu
# Cài đặt chain_type="stuff" để nhồi tất cả các tài liệu tìm được vào prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever, 
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
print("Chain RAG đã được tạo.")

Vectorstore đã được tạo với 3 documents
Chain RAG đã được tạo.


In [ ]:
def chat_with_bot(query):
    result = qa_chain.invoke({"query": query})
    return result['result']

print("\n--- Kiểm tra không có Memory ---")

user_query_1 = "Trần Hưng đạo là ai?"
response_1 = chat_with_bot(user_query_1)
print(f"Bạn: {user_query_1}")
print(f"Bot: {response_1}")

user_query_2 = "Ông ấy có công lao gì?"
response_2 = chat_with_bot(user_query_2)
print(f"Bạn: {user_query_2}")
print(f"Bot: {response_2}")


--- Kiểm tra không có Memory ---
Bạn: Trần Hưng đạo là ai?
Bot: Trần Hưng Đạo là một trong những vị danh tướng kiệt xuất nhất lịch sử Việt Nam.
Bạn: Ông ấy có công lao gì?
Bot: Phạm Ngũ Lão có công rất lớn trong việc xây dựng, bảo vệ, củng cố sức mạnh dưới thời nhà Trần.


In [65]:
from langchain.chains import ConversationalRetrievalChain

# khởi tạo retriever từ vector store (Chroma)
retriever = vectorstore_01.as_retriever(search_kwargs={"k": 5}) # Lấy 5 tài liệu liên quan nhất

# Tạo ConversationalRetrievalChain
# Chain này sẽ tự động quản lý lịch sử trò chuyện và đưa vào ngữ cảnh
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT}
)

print("Chatbot với memory đã sẵn sàng.")

Chatbot với memory đã sẵn sàng.


In [ ]:
def chat_with_bot(query):
    result = conversation_chain.invoke({"question": query})
    return result['answer']

print("\n--- Kiểm tra không có Memory ---")

user_query_1 = "Trần Hưng đạo là ai?"
response_1 = chat_with_bot(user_query_1)
print(f"Bạn: {user_query_1}")
print(f"Bot: {response_1}")

user_query_2 = "Ông có công lao gì?"
response_2 = chat_with_bot(user_query_2)
print(f"Bạn: {user_query_2}")
print(f"Bot: {response_2}")


--- Kiểm tra không có Memory ---
Bạn: Trần Hưng đạo là ai?
Bot: Trần Hưng Đạo là một trong những vị danh tướng kiệt xuất nhất lịch sử Việt Nam.
Bạn: Ông có công lao gì?
Bot: Để trả lời câu hỏi "Ông có những công lao gì?", cần xác định "Ông" là ai. Dựa vào ngữ cảnh, có thể "Ông" là Phạm Ngũ Lão hoặc Hưng Đạo Vương Trần Quốc Tuấn.

*   **Nếu "Ông" là Phạm Ngũ Lão:** Ông có công rất lớn trong việc xây dựng, bảo vệ, củng cố sức mạnh dưới thời nhà Trần.
*   **Nếu "Ông" là Hưng Đạo Vương Trần Quốc Tuấn:** Ông là một trong những vị danh tướng kiệt xuất nhất lịch sử Việt Nam. (Tuy nhiên, ngữ cảnh không nêu cụ thể công lao của ông là gì).


In [75]:
from langchain_core.callbacks import StdOutCallbackHandler

# khởi tạo retriever từ vector store (Chroma)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Tạo ConversationalRetrievalChain
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory, 
    combine_docs_chain_kwargs={"prompt": PROMPT},
    callbacks=[StdOutCallbackHandler()]
)

query = "Việt nam có quan hệ ngoại giao với bao nhiêu nước?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Trần Hưng đạo là ai?
Assistant: Trần Hưng Đạo là một trong những vị danh tướng kiệt xuất nhất lịch sử Việt Nam.
Human: Ông có công lao gì?
Assistant: Để trả lời câu hỏi "Ông có những công lao gì?", cần xác định "Ông" là ai. Dựa vào ngữ cảnh, có thể "Ông" là Phạm Ngũ Lão hoặc Hưng Đạo Vương Trần Quốc Tuấn.

*   **Nếu "Ông" là Phạm Ngũ Lão:** Ông có công rất lớn trong việc xây dựng, bảo vệ, củng cố sức mạnh dưới thời nhà Trần.
*   **Nếu "Ông" là Hưng Đạo Vương Trần Quốc Tuấn:** Ông là một trong những vị danh tướng kiệt xuất nhất lịch sử Việt Nam. (Tuy nhiên, ngữ cảnh không nêu cụ thể công lao của ông là gì).
Human: Việt nam có quan hệ ngoại giao với bao nhiêu nước?
Assistant: Tôi xin lỗi, đoạn văn bản được cu

In [ ]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT}
)

def chat_with_bot(query):
    result = conversation_chain.invoke({"question": query})
    return result['answer']

user_input = "phong trào Cần Vương do ai khởi xướng ?"
response = chat_with_bot(user_input)
print(f"Chatbot: {response}")

user_input = "Nguyễn Ái Quốc thành lập Hội Việt Nam Cách mạng thanh niên tại đâu?"
response = chat_with_bot(user_input)
print(f"Chatbot: {response}")

user_input = "Hội nghị toàn quốc của Đảng họp ở Tân Trào do ai chủ trì?"
response = chat_with_bot(user_input)
print(f"Chatbot: {response}")

Chatbot: Phong trào Cần Vương do vua Hàm Nghi và Tôn Thất Thuyết khởi xướng.
Chatbot: Đoạn văn không đề cập đến địa điểm Nguyễn Ái Quốc thành lập Hội Việt Nam Cách mạng thanh niên.
Chatbot: Đoạn văn bản được cung cấp không có thông tin về hội nghị toàn quốc của Đảng họp ở Tân Trào và ai là người chủ trì.


In [76]:
# In lịch sử trò chuyện để xem Memory có lưu không
print("\n--- Lịch sử trò chuyện trong Memory ---")
for msg in memory.buffer:
    print(f"{msg.type}: {msg.content}")

print("--- Kết thúc kiểm tra Memory ---")


--- Lịch sử trò chuyện trong Memory ---
human: Trần Hưng đạo là ai?
ai: Trần Hưng Đạo là một trong những vị danh tướng kiệt xuất nhất lịch sử Việt Nam.
human: Ông có công lao gì?
ai: Để trả lời câu hỏi "Ông có những công lao gì?", cần xác định "Ông" là ai. Dựa vào ngữ cảnh, có thể "Ông" là Phạm Ngũ Lão hoặc Hưng Đạo Vương Trần Quốc Tuấn.

*   **Nếu "Ông" là Phạm Ngũ Lão:** Ông có công rất lớn trong việc xây dựng, bảo vệ, củng cố sức mạnh dưới thời nhà Trần.
*   **Nếu "Ông" là Hưng Đạo Vương Trần Quốc Tuấn:** Ông là một trong những vị danh tướng kiệt xuất nhất lịch sử Việt Nam. (Tuy nhiên, ngữ cảnh không nêu cụ thể công lao của ông là gì).
human: Việt nam có quan hệ ngoại giao với bao nhiêu nước?
ai: Tôi xin lỗi, đoạn văn bản được cung cấp không chứa thông tin về số lượng quốc gia mà Việt Nam có quan hệ ngoại giao.
human: Việt nam có quan hệ ngoại giao với bao nhiêu nước?
ai: Việt Nam có quan hệ ngoại giao với 160 nước.
human: Việt nam có quan hệ ngoại giao với bao nhiêu nước?
ai: Việt

In [89]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

# 1. Khởi tạo Semantic Retriever (sử dụng Chroma)
semantic_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
print("Semantic Retriever đã được tạo.")

# 2. Khởi tạo Keyword Retriever (sử dụng BM25)
keyword_retriever = BM25Retriever.from_documents(texts, k=5)
print("Keyword Retriever (BM25) đã được tạo.")

# 3. Kết hợp cả hai retriever bằng EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[semantic_retriever, keyword_retriever],
    weights=[0.5, 0.5]
)
print("Ensemble Retriever đã được tạo với trọng số 0.5 cho cả hai.")

Semantic Retriever đã được tạo.
Keyword Retriever (BM25) đã được tạo.
Ensemble Retriever đã được tạo với trọng số 0.5 cho cả hai.


In [90]:
# --- Các ví dụ câu hỏi để kiểm tra ---
test_queries = [
    "Ai là người khởi xướng phong trào Cần Vương?", # Câu hỏi có từ khóa rõ ràng
    "Vì sao thực dân Pháp lại tấn công Đà Nẵng đầu tiên?", # Câu hỏi cần suy luận hoặc tìm thông tin cụ thể
]

for i, query in enumerate(test_queries):
    print(f"\n--- Câu hỏi {i+1}: '{query}' ---")

    print("\n--- Kết quả từ SEMANTIC RETRIEVER ---")
    semantic_docs = semantic_retriever.get_relevant_documents(query)
    for j, doc in enumerate(semantic_docs):
        print(f"  Tài liệu {j+1} (Semantic):")
        print(f"    Nguồn: {doc.metadata.get('source', 'Không rõ')}")
        print(f"    Nội dung (đầu): \"{doc.page_content}...\"") # In 200 ký tự đầu
        print("-" * 20)

    print("\n--- Kết quả từ ENSEMBLE RETRIEVER (Semantic + Keyword) ---")
    ensemble_docs = ensemble_retriever.get_relevant_documents(query)
    for j, doc in enumerate(ensemble_docs):
        print(f"  Tài liệu {j+1} (Ensemble):")
        print(f"    Nguồn: {doc.metadata.get('source', 'Không rõ')}")
        print(f"    Nội dung (đầu): \"{doc.page_content}...\"")
        print("-" * 20)

    print("\n" + "=" * 50) # Dấu phân cách giữa các câu hỏi



--- Câu hỏi 1: 'Ai là người khởi xướng phong trào Cần Vương?' ---

--- Kết quả từ SEMANTIC RETRIEVER ---
  Tài liệu 1 (Semantic):
    Nguồn: data/LSĐ.pdf
    Nội dung (đầu): "trưởng về cơ bản  vẫn theo mô hình cũ,  chậm được đổi mới; tăng  trưởng vẫn chủ yếu dựa vào 
tăng vốn đầu tư và số lượng lao động, chưa dựa nhiều vào  tăng năng suất lao động, ứng dụng 
khoa học-công nghệ. Hội nghị Trung ương 4, khóa XII  (10-2016) đã ra Nghị quyết về tiếp tục đổi 
mới mô hình tăng trưởng, nâng cao chất lượng  tăng trưởng, năng suất lao động và sức cạnh tranh..."
--------------------
  Tài liệu 2 (Semantic):
    Nguồn: data/LSĐ.pdf
    Nội dung (đầu): "trong một bộ phận không nhỏ cán bộ, đảng viên và nhân dân  chưa được khắc phục, làm giảm sức 
chiến đấu của Đảng và lòng tin của nhân dân với  Đảng và chế độ. Trong Đảng đã xuất hiện các ý 
kiến khác nhau về đường lối của Đảng,  chính sách của Nhà nước. Trong xã hội đã phát sinh tâm 
trạng bức xúc đáng lo ngại. Các  phần tử cơ hội, bất mãn trong nư

In [91]:
# Tạo ConversationalRetrievalChain
# Bây giờ, sử dụng ensemble_retriever mới tạo
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=ensemble_retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT}
)

print("Chatbot với memory và Ensemble Retriever đã sẵn sàng.")

Chatbot với memory và Ensemble Retriever đã sẵn sàng.


In [92]:
# Hàm để trò chuyện với bot
def chat_with_bot(query):
    result = conversation_chain.invoke({"question": query})
    return result['answer']

user_input = "phong trào Cần Vương do ai khởi xướng ?"
response = chat_with_bot(user_input)
print(f"Chatbot: {response}")

user_input = "Nguyễn Ái Quốc thành lập Hội Việt Nam Cách mạng thanh niên tại đâu?"
response = chat_with_bot(user_input)
print(f"Chatbot: {response}")

user_input = "Hội nghị toàn quốc của Đảng họp ở Tân Trào do ai chủ trì?"
response = chat_with_bot(user_input)
print(f"Chatbot: {response}")

Chatbot: Phong trào Cần Vương do vua Hàm Nghi và Tôn Thất Thuyết khởi xướng.
Chatbot: Nguyễn Ái Quốc thành lập Hội Việt Nam Cách mạng thanh niên tại Quảng Châu (Trung Quốc).
Chatbot: Hội nghị toàn quốc của Đảng họp ở Tân Trào do lãnh tụ Hồ Chí Minh và Tổng Bí thư Trường Chinh chủ trì.


In [93]:
import gradio as gr

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

# And in Gradio:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
